In [4]:
import pandas as pd
from tqdm import tqdm
import os

In [5]:
def get_csv_paths(folder_path, recursive=False):
    if recursive:
        # Walk through all subdirectories
        file_paths = [os.path.join(root, file) 
                      for root, _, files in os.walk(folder_path) 
                      for file in files if file.endswith('.csv')]
    else:
        # Get files in the root folder only
        file_paths = [os.path.join(folder_path, file) 
                      for file in os.listdir(folder_path) 
                      if file.endswith('.csv')]
    
    return file_paths

In [8]:
paths = get_csv_paths("../data/features/data_human")
paths

['../data/features/data_human/natural-questions_human_features.csv',
 '../data/features/data_human/blogs_human_features.csv',
 '../data/features/data_human/raid_human_features.csv',
 '../data/features/data_human/essays_human_features.csv',
 '../data/features/data_human/writingprompts_human_features.csv',
 '../data/features/data_human/xsum_human_features.csv',
 '../data/features/data_human/tweets_human_features.csv',
 '../data/features/data_human/nyt-comments_human_features.csv',
 '../data/features/data_human/nyt-articles_human_features.csv',
 '../data/features/data_human/reddit_human_features.csv']

In [10]:
for path in tqdm(paths):
    df = pd.read_csv(path)
    total_na = df.isna().sum().sum()
    col_na = df["dependency_distance"].isna().sum()
    print(col_na, col_na/len(df))
    if total_na !=col_na:
        t = input("There are {} NaN values in the dataframe, but only {} in the column 'dependency_distance'. Do you want to drop all NaN values? (y/n)".format(total_na, col_na))
        if t == "y":
            df["dependency_distance"] = df["dependency_distance"].fillna(0)
            df.to_csv(path, index=False)
    else:
        df["dependency_distance"] = df["dependency_distance"].fillna(0)
        df.to_csv(path, index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

0 0.0


 10%|█         | 1/10 [00:02<00:23,  2.62s/it]

66 0.00011443810025817929


 20%|██        | 2/10 [00:25<01:55, 14.39s/it]

...

 80%|████████  | 8/10 [04:21<02:19, 69.52s/it]

0 0.0


 90%|█████████ | 9/10 [04:22<00:48, 48.07s/it]

1 1.5255902508680608e-06


100%|██████████| 10/10 [04:49<00:00, 28.95s/it]


In [11]:
for ds in ["blogs", "essays", "natural_questions", "nyt_articles", "nyt_comments", "raid", "reddit", "tweets", "writingprompts", "xsum"]:
    DATA_HUMAN_PATH = "../data/features/data_human"
    DATA_AI_PATH = f"../data/features/data_ai/{ds}"
    paths = get_csv_paths(DATA_AI_PATH, recursive=True)
    dff = pd.read_csv(f'../data/features/data_human/{ds.replace("_", "-")}_human_features.csv')
    df = pd.read_csv(f'../data/data_human/{ds.replace("_", "-")}_human.csv')
    dff["text"] = df["text"]
    for path in tqdm(paths):
        tmpf = pd.read_csv(path)
        total_na = tmpf.isna().sum().sum()
        col_na = tmpf["dependency_distance"].isna().sum()
        print(f"Total NA: {total_na}, Column NA: {col_na}, {col_na/len(tmpf)}")
        if total_na != col_na:
            t = input("Continue? (y/n): ")
            if t == "y":
                tmpf["dependency_distance"] = tmpf["dependency_distance"].fillna(0)
        else:
            tmpf["dependency_distance"] = tmpf["dependency_distance"].fillna(0)


        data_path = "../data/" + "/".join(path.split("/")[-3:]).replace("_features", "")
        tmp = pd.read_csv(data_path)
        tmpf["text"] = tmp["base_text"]
        tmpf = tmpf.merge(dff, on="text", how="left", suffixes=("", "_base"))
        tmpf.drop(columns=["text"], inplace=True)
        if len(tmp) != len(tmpf):
            print(f"Length mismatch for {path}: {len(tmp)} vs {len(tmpf)}")
            t = input("Continue? (y/n): ")
            if t == "y":
                tmpf.to_csv(path, index=False)
        else:
            tmpf.to_csv(path, index=False)

  0%|          | 0/21 [00:00<?, ?it/s]

Total NA: 2, Column NA: 2, 6.936977558877598e-05


  5%|▍         | 1/21 [00:05<01:45,  5.29s/it]

Total NA: 5, Column NA: 5, 0.00017339436815092245


 10%|▉         | 2/21 [00:09<01:32,  4.85s/it]

...

 90%|█████████ | 19/21 [02:26<00:15,  7.70s/it]

Total NA: 0, Column NA: 0, 0.0


 95%|█████████▌| 20/21 [02:32<00:07,  7.25s/it]

Total NA: 0, Column NA: 0, 0.0


100%|██████████| 21/21 [02:38<00:00,  7.56s/it]
